In [67]:
import csv
import os
import random

import numpy as np
import matplotlib.pyplot as plt

In [68]:
def read_region_data(filename, regions):
    with open(filename, "r") as csvfile:
        reader = csv.reader(csvfile, delimiter=";")
        next(reader) 
        for row in reader:
            region_id, latitude, longitude, avg_pop, driving_perc, avg_m_inc, chargers, traffic = row
            latitude = float(latitude.replace(",", "."))
            longitude = float(longitude.replace(",", "."))
            avg_pop = int(avg_pop)
            driving_perc = float(driving_perc.replace(",", "."))
            avg_m_inc = float(avg_m_inc.replace(",", "."))
            chargers = int(chargers)
            traffic = int(traffic)
            regions.append({
                "region_id": region_id,
                "latitude": latitude,
                "longitude": longitude,
                "avg_drivers": int(avg_pop * driving_perc),
                "avg_income": avg_m_inc,
                "chargers": chargers,
                "traffic": traffic
            })

In [69]:
def read_car_model_data(filename, cars):
    with open(filename, "r") as csvfile:
        reader = csv.reader(csvfile, delimiter=";")
        next(reader)
        for row in reader:
            car_id, autonomy, price = row
            autonomy = int(autonomy)
            price = int(price)
            cars.append({
                "car_id": car_id,
                "autonomy": autonomy,
                "price": price
            })

In [70]:
region_file = "regions.csv"
regions = []
if os.path.exists(region_file):
    read_region_data(region_file, regions)
        
car_file = "cars.csv"
car_models = []
if os.path.exists(car_file):
    read_car_model_data(car_file, car_models)

In [71]:
salaryFluctuation = 0.325
percWillingToSpend = 0.15
probabilityOfBuying = 0.3

In [72]:
def generate_income(avg_income):
    sigma = np.sqrt(np.log(1 + (salaryFluctuation ** 2)))
    mu = np.log(avg_income) - (sigma**2 / 2)
    return np.random.lognormal(mu, sigma)

In [73]:
def affordable_cars(income):
    affordable = []
    for car in car_models:
        if car['price'] <= income * percWillingToSpend:
            affordable.append(car)
    return affordable

In [74]:
def simulate_region(region):
    avg_income = region['avg_income']
    results = {car['car_id']: 0 for car in car_models}
    for _ in range(region['avg_drivers']):
        income = generate_income(avg_income)
        affordable = affordable_cars(income)
        if affordable and random.random() < probabilityOfBuying:
            chosen_car = random.choice(affordable)
            results[chosen_car['car_id']] += 1
    return results

In [75]:
def plot_car_distribution(all_results):
    for region_id, region_result in all_results.items():
        car_ids = [str(car.id) for car in region_result.keys()]
        totals = list(region_result.values())
        plt.figure(figsize=(10, 4))
        bars = plt.bar(car_ids, totals, color=['blue', 'green', 'red', 'cyan', 'magenta', 'yellow'])
        plt.xlabel('Car Model')
        plt.ylabel('Number of Cars Sold')
        plt.title(f'Total Electric Cars Sold in {region_id}')
        plt.xticks(rotation=45)
        plt.tight_layout()
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width() / 2, height, f'{height}', 
                     ha='center', va='bottom', fontsize=8)
        plt.show()

In [76]:
def run():
    total = 0
    all_results = {}
    for region in regions:
        region_result = simulate_region(region)
        all_results[region['region_id']] = region_result
        print(region['region_id'])
        region_cars = ''
        total_cars = 0
        for car in region_result:
            total_cars += region_result[car]
            region_cars += car + ': ' + str(region_result[car]) + ' | '
        region_cars += 'total: ' + str(total_cars)
        total += total_cars
        print(region_cars)
    print('\ntotal: ', total)
    return all_results

In [77]:
run()

aldoar
low_end: 2605 | low_mid_end: 1424 | mid_end: 548 | mid_high_end: 177 | high_end: 17 | top_end: 2 | total: 4773
ramalde
low_end: 441 | low_mid_end: 94 | mid_end: 5 | mid_high_end: 0 | high_end: 0 | top_end: 0 | total: 540
lordelo
low_end: 783 | low_mid_end: 232 | mid_end: 35 | mid_high_end: 6 | high_end: 0 | top_end: 0 | total: 1056
paranhos
low_end: 538 | low_mid_end: 130 | mid_end: 7 | mid_high_end: 1 | high_end: 0 | top_end: 0 | total: 676
centro
low_end: 443 | low_mid_end: 93 | mid_end: 10 | mid_high_end: 1 | high_end: 0 | top_end: 0 | total: 547
bonfim
low_end: 162 | low_mid_end: 46 | mid_end: 5 | mid_high_end: 1 | high_end: 0 | top_end: 0 | total: 214
campanha
low_end: 27 | low_mid_end: 8 | mid_end: 0 | mid_high_end: 0 | high_end: 0 | top_end: 0 | total: 35

total:  7841


{'aldoar': {'low_end': 2605,
  'low_mid_end': 1424,
  'mid_end': 548,
  'mid_high_end': 177,
  'high_end': 17,
  'top_end': 2},
 'ramalde': {'low_end': 441,
  'low_mid_end': 94,
  'mid_end': 5,
  'mid_high_end': 0,
  'high_end': 0,
  'top_end': 0},
 'lordelo': {'low_end': 783,
  'low_mid_end': 232,
  'mid_end': 35,
  'mid_high_end': 6,
  'high_end': 0,
  'top_end': 0},
 'paranhos': {'low_end': 538,
  'low_mid_end': 130,
  'mid_end': 7,
  'mid_high_end': 1,
  'high_end': 0,
  'top_end': 0},
 'centro': {'low_end': 443,
  'low_mid_end': 93,
  'mid_end': 10,
  'mid_high_end': 1,
  'high_end': 0,
  'top_end': 0},
 'bonfim': {'low_end': 162,
  'low_mid_end': 46,
  'mid_end': 5,
  'mid_high_end': 1,
  'high_end': 0,
  'top_end': 0},
 'campanha': {'low_end': 27,
  'low_mid_end': 8,
  'mid_end': 0,
  'mid_high_end': 0,
  'high_end': 0,
  'top_end': 0}}